In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession 
import pyspark.sql.functions as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = 'gs://bkt-dataproc-prod-eu-notebookmatthieubritoantunes/google-merchant-centre/'
filename = 'top_products_gmc_10_2020.parquet'

df_merchant = sqlContext.read.load(path+filename)
df_merchant = (df_merchant.withColumnRenamed('product_category', 'large_product_category')
               .withColumn('product_category', 
                           F.split(F.col('large_product_category'), ' > ').getItem(0))
              )
df_merchant.cache().count()

In [ ]:
#Work carried on on UK data as a first example
df_merchant_uk = df_merchant.filter(F.col('country')=='GB')
df_merchant_uk.cache().count()

# Rising Stars

In [ ]:
(df_merchant_uk.groupBy(F.col('date'),
                        F.col('product_category'),
                        F.col('product_brand'))
 .agg(F.mean('product_rank').cast(IntegerType()).alias('average_rank'))
 .orderBy(F.asc('date'),
          F.asc('product_category'),
          F.asc('average_rank'))
 .filter(F.col('average_rank')<10)
 .show()
)

In [ ]:
(df_merchant_uk.groupBy(F.col('product_brand'),
                        F.col('date'))
 .agg(F.max('product_rank').alias('lowest_rank'),
      F.min('product_rank').alias('highest_rank'),
      F.mean('product_rank').cast(IntegerType()).alias('average_rank'))
 .orderBy(F.asc('product_brand'),
          F.asc('date'))
 .filter(F.col('average_rank')<10)
 .show()
)

# Latest k

# Upcoming Trends